In [57]:
import tensorflow as tf

from keras.layers import Conv2D, AveragePooling2D, BatchNormalization, Input, Dense, Flatten, ZeroPadding2D, MaxPooling2D
from keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Model
from keras.layers.advanced_activations import ReLU
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Add
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.preprocessing import image

In [58]:
# Config variables
# Data images are not of the same size
INPUT_SIZE = [224, 224]
BATCH = 16

In [59]:
# No data augmentation
train_val_image_data_generator = ImageDataGenerator(rescale=1.0/255,
                                            rotation_range=20,
                                            zoom_range=0.2,
                                            width_shift_range=0.2,
                                            height_shift_range=0.2,
                                            shear_range=0.2,
                                            horizontal_flip=True,
                                            fill_mode="nearest",
                                            validation_split=0.2
                                            )

train_data = train_val_image_data_generator.flow_from_directory('./data/',
                                                     target_size = INPUT_SIZE,
                                                     batch_size=BATCH,
                                                     subset='training',
                                                     shuffle=True,
                                                     classes=['with_mask','without_mask']
                                                     )
validate_data = train_val_image_data_generator.flow_from_directory('./data/',
                                                     target_size = INPUT_SIZE,
                                                     batch_size=BATCH,
                                                     subset='validation',
                                                     shuffle=True,
                                                     classes=['with_mask','without_mask']
                                                     )

Found 6043 images belonging to 2 classes.
Found 1510 images belonging to 2 classes.


In [60]:
def identity_block(resnet50_layer_result, num_filter1, num_filter2):
    resnet50_layer_result_skip = resnet50_layer_result

    resnet50_layer_result = Conv2D(num_filter1, kernel_size=(1, 1), strides=(1, 1), padding='valid', kernel_regularizer=l2(0.001))(resnet50_layer_result)
    resnet50_layer_result = BatchNormalization()(resnet50_layer_result)
    resnet50_layer_result = ReLU()(resnet50_layer_result)

    resnet50_layer_result = Conv2D(num_filter1, kernel_size=(3, 3), strides=(1, 1), padding='same', kernel_regularizer=l2(0.001))(resnet50_layer_result)
    resnet50_layer_result = BatchNormalization()(resnet50_layer_result)
    resnet50_layer_result = ReLU()(resnet50_layer_result)

    resnet50_layer_result = Conv2D(num_filter2, kernel_size=(1, 1), strides=(1, 1), padding='valid', kernel_regularizer=l2(0.001))(resnet50_layer_result)
    resnet50_layer_result = BatchNormalization()(resnet50_layer_result)

    resnet50_layer_result = Add()([resnet50_layer_result, resnet50_layer_result_skip])
    resnet50_layer_result = ReLU()(resnet50_layer_result)

    return resnet50_layer_result

def conv_block(resnet50_layer_result, strides, num_filter1, num_filter2):
    resnet50_layer_result_skip = resnet50_layer_result

    resnet50_layer_result = Conv2D(num_filter1, kernel_size=(1, 1), strides=strides, padding='valid', kernel_regularizer=l2(0.001))(resnet50_layer_result)
    resnet50_layer_result = BatchNormalization()(resnet50_layer_result)
    resnet50_layer_result = ReLU()(resnet50_layer_result)

    resnet50_layer_result = Conv2D(num_filter1, kernel_size=(3, 3), strides=(1, 1), padding='same', kernel_regularizer=l2(0.001))(resnet50_layer_result)
    resnet50_layer_result = BatchNormalization()(resnet50_layer_result)
    resnet50_layer_result = ReLU()(resnet50_layer_result)

    resnet50_layer_result = Conv2D(num_filter2, kernel_size=(1, 1), strides=(1, 1), padding='valid', kernel_regularizer=l2(0.001))(resnet50_layer_result)
    resnet50_layer_result = BatchNormalization()(resnet50_layer_result)

    resnet50_layer_result_skip = Conv2D(num_filter2, kernel_size=(1, 1), strides=strides, padding='valid', kernel_regularizer=l2(0.001))(resnet50_layer_result)
    resnet50_layer_result_skip = BatchNormalization()(resnet50_layer_result)

    resnet50_layer_result = Add()([resnet50_layer_result, resnet50_layer_result_skip])
    resnet50_layer_result = ReLU()(resnet50_layer_result)

    return resnet50_layer_result

resnet50_input = Input(shape=INPUT_SIZE + [3])
resnet50_layer_result = ZeroPadding2D(padding=(3, 3))(resnet50_input)

resnet50_layer_result = Conv2D(filters=64, kernel_size=(7, 7), strides=(2, 2))(resnet50_layer_result)
resnet50_layer_result = BatchNormalization()(resnet50_layer_result)
resnet50_layer_result = ReLU()(resnet50_layer_result)
resnet50_layer_result = MaxPooling2D((3, 3), strides=(2, 2))(resnet50_layer_result)

resnet50_layer_result = conv_block(resnet50_layer_result, (1, 1), 64, 256)
resnet50_layer_result = identity_block(resnet50_layer_result, 64, 256)
resnet50_layer_result = identity_block(resnet50_layer_result, 64, 256)

resnet50_layer_result = conv_block(resnet50_layer_result, (2, 2), 128, 512)
resnet50_layer_result = identity_block(resnet50_layer_result, 128, 512)
resnet50_layer_result = identity_block(resnet50_layer_result, 128, 512)
resnet50_layer_result = identity_block(resnet50_layer_result, 128, 512)

resnet50_layer_result = conv_block(resnet50_layer_result, (2, 2), 256, 1024)
resnet50_layer_result = identity_block(resnet50_layer_result, 256, 1024)
resnet50_layer_result = identity_block(resnet50_layer_result, 256, 1024)
resnet50_layer_result = identity_block(resnet50_layer_result, 256, 1024)
resnet50_layer_result = identity_block(resnet50_layer_result, 256, 1024)
resnet50_layer_result = identity_block(resnet50_layer_result, 256, 1024)

resnet50_layer_result = conv_block(resnet50_layer_result, (2, 2), 512, 2048)
resnet50_layer_result = identity_block(resnet50_layer_result, 512, 2048)
resnet50_layer_result = identity_block(resnet50_layer_result, 512, 2048)

resnet50_layer_result = AveragePooling2D((2, 2), padding='same')(resnet50_layer_result)
resnet50_layer_result = Flatten()(resnet50_layer_result)
resnet50_layer_result = Dense(units=2, activation='softmax', kernel_initializer='he_normal')(resnet50_layer_result)

resnet50 = Model(inputs=resnet50_input, outputs=resnet50_layer_result)
resnet50.summary()



Model: "model_14"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_15 (InputLayer)           [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d_14 (ZeroPadding2 (None, 230, 230, 3)  0           input_15[0][0]                   
__________________________________________________________________________________________________
conv2d_761 (Conv2D)             (None, 112, 112, 64) 9472        zero_padding2d_14[0][0]          
__________________________________________________________________________________________________
batch_normalization_761 (BatchN (None, 112, 112, 64) 256         conv2d_761[0][0]                 
___________________________________________________________________________________________

In [24]:
precision = tf.keras.metrics.Precision()
recall = tf.keras.metrics.Recall()
resnet50.compile(
  loss='binary_crossentropy',
  optimizer='SGD',
  metrics=['accuracy', precision, recall]
)

es = EarlyStopping(monitor='val_accuracy', patience=20, verbose=1, mode='auto')

resnet50_r = resnet50.fit(
  train_data,
  validation_data=validate_data,
  epochs=25,
  steps_per_epoch=len(train_data),
  validation_steps=len(validate_data),
  callbacks=[es]
)

Epoch 1/25


Exception ignored in: <function ScopedTFGraph.__del__ at 0x000001B4D70AF4C0>
Traceback (most recent call last):
  File "d:\Anaconda\envs\COMP9444\lib\site-packages\tensorflow\python\framework\c_api_util.py", line 58, in __del__
    self.deleter(self.graph)
KeyboardInterrupt: 


  7/189 [>.............................] - ETA: 2:48 - loss: 16.3082 - accuracy: 0.5580 - precision_3: 0.5580 - recall_3: 0.5580